# Run prediction pipeline for radio-detectable AGN

Jupyter Notebook to run the prediction pipeline presented in Carvajal et al., 2023.

Example code uses data from IR-detected sources in either HETDEX Spring field or Stripe 82.

In [1]:
%matplotlib inline
# Static plots
import numpy as np
from pycaret import classification as pyc
from pycaret import regression as pyr
from joblib import load
import pandas as pd
import global_variables as gv
import global_functions as gf
import os
import subprocess

### Reading data

Select the field to predict sources.

In [2]:
used_field = 'HETDEX'  # 'HETDEX' or 'S82'

In [3]:
file_name_dict      = {'S82': gv.file_S82, 'HETDEX': gv.file_HETDEX}
file_name           = gv.model_path + file_name_dict[used_field]

Check that data files have been downloaded.

In [4]:
gf.download_from_zenodo(file_name_dict[used_field], gv.model_path)

File predicted_rAGN_HETDEX.parquet has been already downloaded


For this notebook, not all columns will be needed. Select those that will be used.

In [5]:
used_cols = ['Z', 'band_num', 'class', 'W1mproPM', 'W2mproPM', 'gmag', 'rmag', 'imag', 'zmag', 'ymag', 'W3mag', 'W4mag', 'Jmag', 'Hmag', 'Kmag', 'LOFAR_detect', 'radio_AGN']

Load file

In [6]:
data_df = pd.read_parquet(file_name, 
                          engine='fastparquet', 
                          columns=used_cols)

Create new columns with colours. Only create colours used by models in pipeline. They are listed in the article.

In [7]:
colours_AGN   = ['g_r', 'r_i', 'r_J', 'i_z', 'i_y', 'z_y', 'z_W2', 'y_J', 'y_W1', 'y_W2', 'J_H', 'H_K', 'H_W3', 'W1_W2', 'W1_W3', 'W3_W4']
colours_radio = ['g_r', 'g_i', 'r_i', 'r_z', 'i_z', 'z_y', 'z_W1', 'y_J', 'y_W1', 'J_H', 'H_K', 'K_W3', 'K_W4', 'W1_W2', 'W2_W3']
colours_z     = ['g_r', 'g_W3', 'r_i', 'r_z', 'i_z', 'i_y', 'z_y', 'y_J', 'y_W1', 'J_H', 'H_K', 'K_W3', 'K_W4', 'W1_W2', 'W2_W3']
    
new_colours   = list(np.unique(colours_AGN + colours_radio + colours_z))

In [8]:
mag_names = {'g': 'gmag', 'r': 'rmag', 'i':'imag', 'z': 'zmag', 'y': 'ymag', 'J': 'Jmag', 'H': 'Hmag', 'K': 'Kmag', 'W1': 'W1mproPM', 'W2': 'W2mproPM', 'W3': 'W3mag', 'W4': 'W4mag'}

In [9]:
for colour in new_colours:
    mag_a_str, mag_b_str = colour.split('_')
    new_col = data_df.loc[:, mag_names[mag_a_str]] - data_df.loc[:, mag_names[mag_b_str]]
    data_df[colour] = new_col

Load models

Check that model files have been downloaded.

In [10]:
gf.download_from_zenodo(gv.AGN_gal_model + '.pkl', gv.model_path)
gf.download_from_zenodo(gv.cal_AGN_gal_model, gv.model_path)
gf.download_from_zenodo(gv.radio_model + '.pkl', gv.model_path)
gf.download_from_zenodo(gv.cal_radio_model, gv.model_path)
gf.download_from_zenodo(gv.full_z_model + '.pkl', gv.model_path)

File classification_AGN_galaxy.pkl has been already downloaded
File cal_classification_AGN_galaxy.joblib has been already downloaded
File classification_radio_detection.pkl has been already downloaded
File cal_classification_radio_detection.joblib has been already downloaded
File regression_redshift.pkl has been already downloaded


In [11]:
AGN_gal_clf           = pyc.load_model(gv.model_path + gv.AGN_gal_model)  #
cal_AGN_gal_clf       = load(gv.model_path + gv.cal_AGN_gal_model)  # calibrated model
radio_det_AGN_clf     = pyc.load_model(gv.model_path + gv.radio_model)
cal_radio_det_AGN_clf = load(gv.model_path + gv.cal_radio_model)  # calibrated model
redshift_reg_rAGN     = pyr.load_model(gv.model_path + gv.full_z_model)

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


#### Run prediction models

Run models over all sources in dataset. The user can select, afterwards, sources that would be predicted as radio-detectable AGN (or any other combination of predictions).
The following steps take some time to run.

Classify between AGN and galaxies.

In [12]:
data_df = gf.predict_AGN_gal(data_df, 
                             AGN_gal_clf,
                             cal_AGN_gal_clf,
                             gv.AGN_thresh,
                             gv.cal_AGN_thresh)

Classify between radio-detectable and non radio-detectable sources.

In [13]:
data_df = gf.predict_radio_det(data_df,
                               radio_det_AGN_clf,
                               cal_radio_det_AGN_clf,
                               gv.radio_thresh,
                               gv.cal_radio_thresh)

Predict photometric redshifts.

In [14]:
data_df = gf.predict_z(data_df, 
                       redshift_reg_rAGN)

Display ten first radio-detected AGN correctly predicted as such in data frame.

In [18]:
filter_data = np.array(data_df.loc[:, 'class'] == 1) & np.array(data_df.loc[:, 'pred_class_cal'] == 1) & np.array(data_df.loc[:, 'LOFAR_detect'] == 1) & np.array(data_df.loc[:, 'pred_radio_cal_AGN'] == 1)

In [19]:
display(data_df.loc[filter_data][:10])

,Z,band_num,class,W1mproPM,W2mproPM,gmag,rmag,imag,zmag,ymag,W3mag,W4mag,Jmag,Hmag,Kmag,LOFAR_detect,radio_AGN,H_K,H_W3,J_H,K_W3,K_W4,W1_W2,W1_W3,W2_W3,W3_W4,g_W3,g_i,g_r,i_y,i_z,r_J,r_i,r_z,y_J,y_W1,y_W2,z_W1,z_W2,z_y,pred_class,Score_AGN,Prob_AGN,pred_class_cal,pred_radio_AGN,Score_radio_AGN,Prob_radio_AGN,pred_radio_cal_AGN,pred_Z_rAGN
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16456,0.846,8,1.0,18.337000,18.363001,23.299999,21.890100,21.287201,20.959600,20.519300,16.670000,14.620,17.450001,17.240,16.590,1,1,0.650000,0.570,0.210001,-0.080000,1.970000,-0.026001,1.667000,1.693001,2.050,6.629999,2.012798,1.409899,0.767900,0.327600,4.440100,0.602900,0.930500,3.069300,2.182301,2.156300,2.622601,2.596600,0.440300,1,0.500016,0.657091,1,0,0.438512,0.410227,1,0.7586
27470,0.736,9,1.0,17.676001,17.408001,20.004499,19.639299,19.607401,19.268499,19.199200,16.670000,14.620,17.450001,17.240,16.590,1,1,0.650000,0.570,0.210001,-0.080000,1.970000,0.268000,1.006001,0.738001,2.050,3.334499,0.397099,0.365200,0.408201,0.338902,2.189299,0.031898,0.370800,1.749199,1.523199,1.791199,1.592499,1.860498,0.069300,1,0.500121,0.988672,1,0,0.310507,0.312719,1,0.7748
40804,1.078,9,1.0,17.311001,16.582001,18.525999,18.507601,18.457300,18.362700,18.208500,15.373000,14.180,17.450001,17.240,16.590,1,1,0.650000,1.867,0.210001,1.217000,2.410000,0.729000,1.938001,1.209001,1.193,3.152999,0.068699,0.018398,0.248800,0.094601,1.057600,0.050301,0.144901,0.758499,0.897499,1.626499,1.051699,1.780699,0.154200,1,0.500122,0.989167,1,0,0.400925,0.382640,1,1.0918
44825,0.877,9,1.0,18.306000,18.175001,19.906601,19.838699,19.618500,19.524599,19.334000,16.670000,14.620,17.450001,17.240,16.590,1,1,0.650000,0.570,0.210001,-0.080000,1.970000,0.130999,1.636000,1.505001,2.050,3.236601,0.288101,0.067902,0.284500,0.093901,2.388699,0.220200,0.314100,1.883999,1.028000,1.158998,1.218599,1.349598,0.190599,1,0.500121,0.988471,1,0,0.501761,0.455192,1,0.9805
45506,0.643,12,1.0,15.579000,15.040000,17.124001,17.214199,17.213301,17.390400,17.359301,14.250999,13.386,17.230000,17.105,16.527,1,1,0.577999,2.854,0.125000,2.276001,3.141001,0.539001,1.328001,0.789001,0.865,2.873001,-0.089300,-0.090199,-0.146000,-0.177099,-0.015800,0.000898,-0.176201,0.129301,1.780300,2.319301,1.811399,2.350400,0.031099,1,0.500122,0.988938,1,0,0.875211,0.726470,1,0.6672
53799,0.712,9,1.0,18.170000,18.677999,21.766199,21.311600,20.481701,19.868799,19.801800,16.670000,14.620,17.450001,17.240,16.590,1,1,0.650000,0.570,0.210001,-0.080000,1.970000,-0.507999,1.500000,2.007999,2.050,5.096199,1.284498,0.454599,0.679901,0.612902,3.861599,0.829899,1.442801,2.351799,1.631800,1.123800,1.698799,1.190800,0.066999,0,0.499981,0.349309,1,0,0.508161,0.459659,1,0.7047
58134,1.639,9,1.0,19.097000,18.287001,21.199100,21.056299,20.669300,20.693300,20.603001,16.670000,14.620,17.450001,17.240,16.590,1,1,0.650000,0.570,0.210001,-0.080000,1.970000,0.809999,2.427000,1.617001,2.050,4.529100,0.529800,0.142801,0.066299,-0.024000,3.606298,0.386999,0.362999,3.153000,1.506001,2.316000,1.596300,2.406300,0.090300,1,0.500120,0.988100,1,0,0.453618,0.421117,1,1.6791
62617,3.193,9,1.0,19.868000,19.752001,21.003599,20.839701,20.900000,20.761200,20.644501,16.670000,14.620,17.450001,17.240,16.590,1,1,0.650000,0.570,0.210001,-0.080000,1.970000,0.115999,3.198000,3.082001,2.050,4.333599,0.103600,0.163898,0.255499,0.138800,3.389700,-0.060299,0.078501,3.194500,0.776501,0.892500,0.893200,1.009199,0.116699,1,0.500119,0.987949,1,0,0.306487,0.309471,1,2.8276
63673,0.906,9,1.0,18.917000,18.932001,22.117001,22.021601,21.557800,20.837400,20.292400,16.670000,14.620,17.450001,17.240,16.590,1,1,0.650000,0.570,0.210001,-0.080000,1.970000,-0.015001,2.247000,2.262001,2.050,5.447001,0.559200,0.095400,1.265400,0.720400,4.571600,0.463800,1.184200,2.842400,1.375401,1.360399,1.920401,1.905399,0.545000,1,0.500105,0.979736,1,0,0.606100,0.526861,1,0.9320
